In [10]:
import pandas as pd
import numpy as np

In [12]:
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from keras.models import Model
from keras.layers import Add,Input, Dense, Concatenate, Reshape, Dropout
from keras.layers import Flatten,Activation,LeakyReLU,PReLU,Lambda,LSTM,GRU
from keras.layers import Conv1D,GlobalAveragePooling1D,GlobalMaxPooling1D,TimeDistributed
from keras.layers.wrappers import Bidirectional
from keras.layers.embeddings import Embedding
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.models import load_model
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras import backend as K
from keras.utils import multi_gpu_model
import keras

#进行配置，每个GPU使用60%上限现存
config = tf.ConfigProto()
# os.environ["CUDA_VISIBLE_DEVICES"] = "4,5,6,7"
# config.gpu_options.per_process_gpu_memory_fraction = 0.4 # 每个GPU现存上届控制在60%以内
session = tf.Session(config=config)

# 设置session
KTF.set_session(session )

Using TensorFlow backend.


In [2]:
# pred_lbl=pd.read_csv('../huawei/pred_nn_try_1_0_5800.csv',header=None)

In [3]:
# pred_lbl_2=pd.read_csv('../huawei/pred_lgb_guess.csv',header=None)

In [23]:
age_train['age_group'].value_counts()/age_train.shape[0]

3    0.200
2    0.200
4    0.175
1    0.175
5    0.125
0    0.125
Name: age_group, dtype: float64

In [24]:
age_train['age_group'].value_counts()

3    800000
2    800000
4    700000
1    700000
5    500000
0    500000
Name: age_group, dtype: int64

In [59]:
# pred_yc_1=pd.read_csv('./pred_df_final.csv',header=None)
pred_yc_2=pd.read_csv('./pred_df_new.csv',header=None)
pred_yc_3=pd.read_csv('./pred_df_new_999.csv',header=None)
# pred_yc_4=pd.read_csv('./pred_nn_try_2_0.csv',header=None)

In [60]:
result=pred_yc_2+pred_yc_3

In [48]:
# result=pd.read_csv('./pred_df_new_999.csv',header=None)

In [61]:
data_dir='/home/uniml/work/hw/dataset/'
temp_save_dir='./temp_data/'
age_train_file_path=data_dir+'age_train.csv'
age_test_file_path=data_dir+'age_test.csv'
app_info_file_path=data_dir+'app_info.csv'
user_app_actived_file_path=data_dir+'user_app_actived.csv'
user_basic_info_file_path=data_dir+'user_basic_info.csv'
user_behavior_info_file_path=data_dir+'user_behavior_info.csv'
user_app_usage_file_path=data_dir+'user_app_usage.csv'

In [62]:
age_train=pd.read_csv(age_train_file_path,header=None,names=['uId','age_group'])
age_train.age_group=age_train.age_group.apply(lambda x:x-1)
age_train.age_group=age_train.age_group.astype('category')
age_test=pd.read_csv(age_test_file_path,header=None,names=['id'])

In [63]:
result=np.argmax(result.values,axis=1)+1
result=pd.DataFrame(result)
submission=pd.concat([age_test,result],axis=1)
submission.rename(columns={0:'label'},inplace=True)
submission.to_csv('submission.csv',index=False)
print(submission.label.value_counts()/len(submission))

4    0.329146
3    0.320890
5    0.131278
2    0.102071
1    0.066500
6    0.050115
Name: label, dtype: float64


In [64]:
submission.head()

,id,label
0,6708326,3
1,6708314,3
2,6708313,3
3,6708284,4
4,6708272,3


In [9]:
base_dir = './dataset/'
app_actived_df = pd.read_csv(base_dir+'user_app_actived.csv', header=None, names=['uId','appId'],
                             dtype={'uId':np.uint32,'appId':str})
app_actived_df.head()

,uId,appId
0,1000110,a001048#a003072#a004443#a006024#a007087#a00743...
1,1000542,a001010#a00158#a001671#a002450#a003484#a003577...
2,1000866,a001048#a00108#a004622#a007104#a0075
3,1001028,a001012#a001055#a001062#a001275#a001403#a00158...
4,1001190,a001012#a00107#a001304#a001403#a001533#a001679...


In [10]:
train_id_set = pd.read_csv(base_dir+'age_train.csv',header=None, names=['uId', 'age_group'])
test_id_set = pd.read_csv(base_dir+'age_test.csv', header=None, names=['uId'])

In [13]:
apps = app_actived_df[['appId']].values
vocab_size = 10010
encoded_docs = [one_hot(app[0], vocab_size, split='#') for app in apps]
max_length = 150
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
padded_docs = np.array(padded_docs)
padded_docs_uId = pd.concat([app_actived_df[['uId']],pd.DataFrame(padded_docs)], axis=1)
train_apps_input = train_id_set[['uId']].merge(padded_docs_uId, on=['uId'], how='left')
test_apps_input = test_id_set[['uId']].merge(padded_docs_uId, on=['uId'], how='left')
train_apps_input.fillna(0,inplace=True)
test_apps_input.fillna(0,inplace=True)
_ = train_apps_input.pop('uId')
_ = test_apps_input.pop('uId')
print(train_apps_input.shape,test_apps_input.shape)

(4000000, 150) (1000000, 150)


In [14]:
train_apps_input.head()

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,2710.0,991.0,9366.0,7368.0,7448.0,788.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1895.0,5038.0,7084.0,365.0,429.0,7128.0,9377.0,8203.0,4591.0,1338.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5353.0,5038.0,4775.0,9378.0,655.0,2662.0,559.0,2364.0,1257.0,7177.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1895.0,5055.0,5038.0,9165.0,1752.0,4389.0,8217.0,9801.0,365.0,1621.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5038.0,365.0,7236.0,6819.0,4591.0,2364.0,1596.0,1257.0,2674.0,6208.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
